# **Stable Diffusion** 🎨
*...using '🧨diffusers'*

Stable Diffusionは、[CompVis](https://github.com/CompVis)、[Stability AI](https://stability.ai/)、[LAION](https://laion.ai/)の研究者やエンジニアが開発したtext-to-imageのdiffusion modelです。[LAION-5B](https://laion.ai/blog/laion-5b/)データベースのサブセットの512x512の画像で学習しています。このモデルは圧縮されたCLIP ViT-L/14テキストエンコーダーを使用し、文字入力からモデルを条件付けます。860MのUNetと123Mのテキストエンコーダを持つこのモデルは、比較的軽量で、多くの個人向けGPUで動作します。詳しくは[model card](https://huggingface.co/CompVis/stable-diffusion)をご覧ください。

このノートブックでは、🤗Hugging Face[🧨Diffusers library](https://github.com/huggingface/diffusers)を使って、Stable Diffusionを使用する方法を紹介します。

それでは始めてみましょう。

## 1. `StableDiffusionPipeline`の使い方

StableDiffusionがどのように動作するのか説明する前に、少し動かしてみましょう🤗。

このセクションでは、簡単にテキストから画像の推論をする方法をご紹介します。

### 前準備

はじめに、このノートブックにGPUが使用されているか確認してください。推論がより速くなります。

In [ ]:
!nvidia-smi

次に、`diffusers`、`scipy`、`ftfy`、`transformers`をインストールします。`accelerate`はロード時間の短縮に用いられます。

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
!python -m pip install diffusers==0.11.1
!python -m pip install transformers scipy ftfy accelerate

### Stable Diffusion Pipeline

`StableDiffusionPipeline`はend-to-endの推論パイプラインで、テキストから画像を簡単に生成するために用いられます。

まず、モデル中のすべてのコンポーネントの事前学習済みの重みをロードします。このノートブックでは、Stable Diffusion バージョン1.4([CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4))を使用します。他にもバリエーションがあるので、ぜひお試しください。

* [runwayml/stable-diffusion-v1-5](https://huggingface.co/runwayml/stable-diffusion-v1-5)
* [stabilityai/stable-diffusion-2-1-base](https://huggingface.co/stabilityai/stable-diffusion-2-1-base)
* [stabilityai/stable-diffusion-2-1](https://huggingface.co/stabilityai/stable-diffusion-2-1) このバージョンは768x768の画像を生成しますが、他のバージョンは512x512で動作します。

モデルID[CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4)に加えて、`from_pretrained`にも特定の`revision`と`torch_dtype`を渡しています。

無料版のGoogle Colab上でもStable Diffusionを実行できるようにしたいので、班制度ブランチからウェイトをロードしています。[`fp16`](https://huggingface.co/CompVis/stable-diffusion-v1-4/tree/fp16)を使用し、`torch_dtype=torch.float16`を渡すことで、float16精度の重みを期待するようにディフューザーに伝えます。

可能な限り高い精度を確保したい場合は、メモリ使用量が増えることを覚悟のうえで、`torch_dtype=torch.float16`を削除してください。

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

次に、パイプラインをGPUに移して、インターフェイスを高速化しましょう。

In [ ]:
pipe = pipe.to("cuda")

画像を生成するための準備が整いました。

In [ ]:
prompt = "a phoyograph of an astronaut riding a horse"
image = pipe(prompt).images[0]

image.save(f"astronaut_rides_horse.png")

image

上記のセルを複数回実行すると、毎回異なる画像が出力されます。出力が変化することが望ましくない場合、パイプラインにseed値を渡すことができます。同じseedを使うことで、同じ画像が出力されます。

In [ ]:
import torch

generator = torch.Generator("cuda").manual_seed(1024)

image = pipe(prompt, generator=generator).images[0]

image

引数`num_inference_steps`を使って推論ステップ数を変更することができます。一般的に、ステップ数が多いほど結果はよくなります。Stable Diffusionは最新のモデルの1つであり、比較的少ないステップ数でもうまく機能するため、デフォルトの50を使用することをお勧めします。もしより速く結果を得たいのであれば、より小さな数字を指定しましょう。

次のセルは、前と同じシードを使用していますが、より少ないステップ数です。馬の頭やヘルメットのような細部は、前の画像と比べて明確に表現されていないことに注意してください。

In [ ]:
import torch

generator = torch.Generator("cuda").manual_seed(1024)

image = pipe(prompt, num_inference_steps=15, generator=generator).images[0]

image

パイプラインコールのもう一つのパラメータは`guidance_scale`です。これは、条件付信号（今回はテキスト）及び全体的なサンプルの品質への準拠を高めるための方法です。簡単に言うと、分類氏を使わないガイダンスは、よりプロンプトにマッチするように生成を強制します。`7`や`8.5`のような数字は良い結果をもたらしますが、非常に大きな数字を使用した場合、画像はよく見えるかもしれませんが、多様性に欠けることになります。

このパラメータの技術的な詳細についてはこのノートブックの[最後のセクション](https://colab.research.google.com/drive/1ALXuCM5iNnJDNW5vqBm5lCtUQtZJHN2f?authuser=1#scrollTo=UZp-ynZLrS-S)で説明します。

同じプロンプトに対して複数の画像を生成するには、同じプロンプトを複数回繰り返したリストを使用するだけです。先ほど使った文字列の代わりに、このリストをパイプラインに送ります。

まず、関数のグリッドを表示するためのヘルパー関数を書いてみましょう。以下のセルを実行すると`image_grid`関数が作成されます。どのように行われているか興味がある方はコードを精読してください。

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols
    
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols * w, rows * h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

これで、3つのプロンプトのリストでパイプラインを実行すれば、グリッド画像が生成できるようになりました。

In [ ]:
num_images = 3
prompt = ["a photograph of an astronaut riding a horse"] * num_images

images = pipe(prompt).images

grid = image_grid(images, rows=1, cols=3)
grid

`n × m`の画像を出力するには次のようにします。

In [ ]:
num_cols = 3
num_rows = 4

prompt = ["a photograph of an astronaut riding a horse"] * num_cols

all_images = []
for i in range(num_rows):
    images = pipe(prompt).images
    all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

### 正方形でない画像の生成

Stable Diffusionは、デフォルトで`512x512`ピクセルの画像を作成します。しかし、`height`や`width`の引数を使って、長方形の画像を作成することもできます。

以下は、適切なサイズを選択するためのいくつかの推奨事項です。
- `height`と`width`がともに`8`の倍数であることを確認してください。
- 512を下回ると、低品質の画像になる可能性があります。
- 縦横共に512を超えると、画像領域が繰り返されます（全体的なまとまりが失われます）。
- 長方形の画像を作成するときは、1つの次元に`512`を、もう1つの次元にそれよりも大きな値を使用するのが効果的です。

In [ ]:
prompt = "a photograph of an astronaut riding a horse"

image = pipe(prompt, height=512, width=640).images[0]
image

## 2. Stable Diffusionとは

さて、Stable Diffusion👩‍🎓の論理的な部分について説明します。

Stable Diffusionは、[High-Resolution Image Synthesis with Latent Diffusion Models](https://arxiv.org/abs/2112.10752)で提案された**Latent Diffusion**という特殊なDiffusionモデルに基づいています。

一般的なDiffusionモデルは、ランダムなガウシアンノイズを段階的に除去するように訓練された機械学習システムで、画像などの目的のサンプルに到達するために使用されます。より詳細な仕組みについては、こちらの[Colab](https://colab/research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb)をご覧ください。

Diffusionモデルは、画像データの生成において、最先端の結果を達成することが示されています。しかし、Diffusionモデルの欠点は、逆ノイズ処理に時間がかかることです。また、これらのモデルはピクセル空間で動作するため、多くのメモリを消費し、高解像度の画像を生成する場合には不当に高価になります。そのため、これらのモデルを訓練し、さらに推論に使用することは困難です。



<br>

Latent Diffusionは、実際にピクセル空間を使用する代わりに、低次元の潜在空間上で拡散プロセスを適用することにより、メモリと計算の複雑さを軽減することができます。これが標準的なDiffusionモデルとLatent Diffusionモデルの主な違いです。**Latent Diffusionでは、モデルは画像の潜伏（圧縮）表現を生成するように訓練されます。**

Latent Diffusionについては、3つの主なコンポーネントがあります。

1. オートエンコーダ（VAE）
2. [U-Net](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb#scrollTo=wW8o1Wp0zRkq)
3. テキストエンコーダ（[CLIP's Text Encoder](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel)など）

**1. オートエンコーダ (VAE)**

VAEモデルはエンコーダとデコーダの二つの部分があります。エンコーダは画像を低次元の潜在表現に変換し、U-Netの入力として使用されます。デコーダは、逆に潜在表現を画像に変換します。

LatentDiffusionでは、エンコーダは各ステップでより多くのノイズを適用する順拡散プロセスのために、画像の潜像表現を得るために使用されます。推論では、逆拡散処理で生成された潜在表現を、デコーダを用いて画像に戻します。後述するように、推論時に必要なのはVAEデコーダのみです。

**2. U-Net**

U-NetはResNetブロックで構成されたエンコーダとデコーダでできています。
エンコーダは画像を低解像度に圧縮し、デコーダは低解像度表現をノイズが除去された高解像度表現に復元します。
具体的には、U-Netの出力は、ノイズの除去された画像を推論するために使用するノイズ残差を予測します。

U-Netがダウンサンプリング中に重要な情報を失うのを防ぐため、通常、エンコーダのダウンサンプリングResNetとデコーダのアップサンプリングResNetの間にショートカットが接続されます。
さらに、StableDiffusion U-Netは、相互アテンション層を介して、text-embeddingに出力を条件づけることができる。相互アテンション層は、U-Netのエンコーダとデコーダの両方に、通常ResNetブロックの間に追加されます。

**3. テキストエンコーダ**

テキストエンコーダは、入力されたプロンプト、例えば「An astronout riding a horse」をU-Netが理解できるように変換する役割を担っています。これは通常、入力トークンシーケンスを潜在的なテキスト埋め込みシーケンスにマッピングする単純な変換器ベースのエンコーダです。

[Imagen](https://imagen.research.google/)に影響されたStableDiffusionは、学習中にテキストエンコーダを訓練せず、単にCLIPの訓練済みテキストエンコーダ、[CLIPTextModel](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel)を使用しています。

**なぜLatentDiffusionは高速で効率的なのか**

LatentDiffusionモデルのU-Netは低次元空間で動作するため、ピクセル空間のDiffusionモデルと比較してメモリや計算量を大幅に削減することができます。例えば、StableDiffusionで使用されるオートエンコーダは、縮小率が8であり、これは、形状が`(3, 512, 512)`の画像が潜像空間では`(3, 64, 64)`になり、`8 × 8 = 64`倍ものメモリを削減できることを意味しています。

これが、16GBのColabGPUでも512x512の画像を高速に生成できる秘訣です。

**推論中のStableDiffusion**

このように、推論的な流れを示すことで、推論においてモデルがどのように機能するのかを詳しくみていきましょう。

<p align="left">
<img src="https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/stable_diffusion.png" alt="sd-pipeline" width="500">

StableDiffusionモデルは、潜像的なseedとテキストプロンプトの両方を入力として受け取ります。潜在seedは$64 \times 64$サイズのランダムな潜在表現を生成するために用いられ、テキストプロンプトはCLIPのテキストエンコーダによって$77 \times 768$サイズのテキスト埋め込みに変換されます。

次にU-Netは、テキスト埋め込みを条件としながら、ランダムな潜像表現を繰り返しノイズ除去します。U-Netの出力はノイズ残差であり、スケジューラアルゴリズムによってノイズ除去された潜像表現を計算するために使用されます。この計算には多くの異なるスケジューラアルゴリズムを使用することができ、それぞれに長所と短所があります。StableDiffusionでは、以下のいずれかを主に使用します。

- [PNDM scheduler](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_pndm.py) (デフォルト).
- [K-LMS scheduler](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_lms_discrete.py).
- [Heun Discrete scheduler](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_heun_discrete).
- [DPM Solver Multistep scheduler](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_dpmsolver_multistep.py).このスケジューラは、より少ないステップです触れた品質を実現することができます。デフォルトの50ステップではなく、25ステップで試してみてください。

スケジューラアルゴリズムの機能に関する理論は、このノートブックの範囲外ですが、要するに、以前のノイズ表現とノイズ残差から、予測ノイズ除去画像表現を計算することを覚えておくと良いでしょう。より詳細な情報については、[Elucidating the Design Space of Diffusion-Based Generative Models](https://arxiv.org/abs/2206.00364)を参照してください。

このノイズ除去処理を約50回繰り返すことで、より良い潜像表現を段階的に取得します。潜像表現は、変分オートエンコーダのデコーダ部によって復号化されます。



このように、「Latent Diffusion」と「Stable Diffusion」を簡単に紹介した後は、「🤗Hugging Face Diffusers」の高度な活用法を見ていきましょう!

## 3. `diffusers`を用いた独自の推論パイプラインの書き方

最後に、`diffusers`を使ってカスタムディフュージョンパイプラインを作成する方法について紹介します。これは、システムの特定の機能をもう少し深く掘り下げたり、特定のコンポーネントを入れ替える可能性がある場合に非常に便利です。

このセクションでは、異なるスケジューラ、即ち [今回のPR](https://github.com/huggingface/diffusers/pull/185#pullrequestreview-1074247365)で追加された[Katherine Crowson's](https://github.com/crowsonkb)K-LMSスケジューラでStable Diffusionを使用する方法を示します。

それでは、`StableDiffusionPipeline`を自分たちでどのように書いたか順を追ってみていきましょう。

まず、関係する個々のモデルをロードすることから始めます。

In [ ]:
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

[訓練済みモデル](https://huggingface.co/CompVis/stable-diffusion-v1-3-diffusers/tree/main)には完全なdiffusionパイプラインをセットアップするために必要なすべてのコンポーネントが含まれています。これらはのコンポーネントは以下のフォルダに格納されています。

- `text_encoder`：Stable DiffusionはCLIPを使用していますが、他のDiffusionモデルでは`BERT`等の他のエンコーダを使用することがあります。
- `tokenizer`：これは`text_encoder`モデルが使用するものと一致する必要があります。
- `scheduler`：訓練中に画像にノイズを徐々に追加していく際に使用するスケジューリングアルゴリズムです。
- `unet`：入力の潜在的な表現を生成するために使用されるモデルです。
- `vae`：潜在表現を実際の画像にデコードするために使用するオートエンコーダモジュール

`from_pretrained`の引数`subfolder`を使って、保存されたフォルダを参照することでコンポーネントをロードすることができます。

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

# 1. Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# 2. Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

次に予め用意されたスケジューラを読み込む代わりにK-LMSのスケジューラを使うようにします。

In [ ]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")

次にモデルをGPUに移動します。

In [ ]:
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device) 

ここで、画像生成に使用するパラメータを定義します。

なお、`guidance_scale`は[Imagen paper](https://arxiv.org/pdf/2205.11487.pdf)の式(2)のガイダンス重み`w`に類似して定義されています。`guidance_scale==1`は、分類子を使わないガイダンスを行うことに相当します。ここでは、前回と同様に7.5としました。

これまでの例とは対照的に、より明確な画像を得るために`num_inference_steps`を100に設定しています。

In [ ]:
prompt = ["a photograph of an astronaut riding a horse"]

height = 512                        # default height of Stable Diffusion
width = 512                         # default width of Stable Diffusion

num_inference_steps = 100            # Number of denoising steps

guidance_scale = 7.5                # Scale for classifier-free guidance

generator = torch.manual_seed(32)   # Seed generator to create the inital latent noise

batch_size = 1

はじめに、プロンプトのtext_embeddingsを取得します。これらのembeddingsはUNetモデルの条件付けに使われます。

In [ ]:
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

また、分類木なしガイダンスのための無条件のテキスト埋め込みを取得します。これは、空のテキストに対する埋め込みだけです。これらは条件付きの`text_embeddings`(`batch_size`と`seq_length`)と同じ形状である必要があります。

In [ ]:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
    uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]   

分類木を使わないガイダンスのためには、2つのフォワードパスを行う必要があります。一つは条件付き入力(`text_embeddings`)で、もう1つは無条件の埋め込み(`uncond_embeddings`)です。実際には、2回のフォワードパスを避けるために、両者を1つのバッチに連結することができます。

In [ ]:
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

初期ランダムノイズを生成します。

In [ ]:
latents = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
    generator=generator,
)
latents = latents.to(torch_device)

In [ ]:
latents.shape

クールな$64\times64$が想定されます。この潜在表現（純粋なノイズ）を、モデルは後で`512x512`の画像に変換することになります。

次に、選択した`num_inference_steps`でスケジューラを初期化します。これにより、ノイズ除去に使用される`sigmas`と正確な時間ステップ値が計算されます。

In [ ]:
scheduler.set_timesteps(num_inference_steps)

K-LMSスケジューラは`latents`にその`sigma`値を掛ける必要があります。それではやってみましょう。

In [ ]:
latents = latents * scheduler.init_noise_sigma

デノイジングループを描く準備が整いました。

In [ ]:
from tqdm.auto import tqdm
from torch import autocast

for t in tqdm(scheduler.timesteps):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    latent_model_input = torch.cat([latents] * 2)

    latent_model_input = scheduler.scale_model_input(latent_model_input, t)

    # predict the noise residual
    with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

    # perform guidance
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler.step(noise_pred, t, latents).prev_sample

ここで、`vae`を使って、生成された`latents`を画像に戻すデコードを行います。

In [ ]:
# scale and decode the image latents with vae
latents = 1 / 0.18215 * latents

with torch.no_grad():
    image = vae.decode(latents).sample

そして最後に、画像を表示したり保存したりできるように、PILに変換してみましょう。

In [ ]:
image = (image / 2 + 0.5).clamp(0, 1)
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0]

これで、独自のパイプラインを構築したり、diffuserのコンポーネントを好きなように使用するためのすべてのピースが揃いました🔥。